Classification Problem

IMPORING MODULE

In [41]:
import pandas as pd
import numpy as np 

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler 

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.metrics import accuracy_score

In [42]:
data = pd.read_csv("train.csv") 
test = pd.read_csv("test.csv")
pid = test["Patient Id"]

Removing columns which will not impact out target

In [43]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22083 entries, 0 to 22082
Data columns (total 45 columns):
 #   Column                                            Non-Null Count  Dtype  
---  ------                                            --------------  -----  
 0   Patient Id                                        22083 non-null  object 
 1   Patient Age                                       20656 non-null  float64
 2   Genes in mother's side                            22083 non-null  object 
 3   Inherited from father                             21777 non-null  object 
 4   Maternal gene                                     19273 non-null  object 
 5   Paternal gene                                     22083 non-null  object 
 6   Blood cell count (mcL)                            22083 non-null  float64
 7   Patient First Name                                22083 non-null  object 
 8   Family Name                                       12392 non-null  object 
 9   Father's name    

In [44]:
data["Genetic Disorder"].mode()[0]


'Mitochondrial genetic inheritance disorders'

In [45]:
data.drop(["Patient Id","Institute Name", "Patient First Name", "Family Name", "Father's name"],axis=1,inplace=True) 

In [46]:
test.drop(["Patient Id","Institute Name", "Patient First Name", "Family Name", "Father's name"],axis=1,inplace=True) 

In [47]:
data["Symptom 3"].isnull()                                                    

0        False
1        False
2        False
3        False
4        False
         ...  
22078    False
22079    False
22080    False
22081    False
22082    False
Name: Symptom 3, Length: 22083, dtype: bool

In [48]:
for i in data.columns:
    t = data[i].isnull()
    for j in range(len(data[i])):
        if t[j]==True: 
            data[i][j]=data[i].mode()[0]
             

<ipython-input-48-1d47fb9433fc>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[i][j]=data[i].mode()[0]


In [49]:
for i in test.columns:
    t = test[i].isnull()
    for j in range(len(test[i])):
        if t[j]==True: 
            test[i][j]=test[i].mode()[0]
           

<ipython-input-49-d0ad966f3dfe>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test[i][j]=test[i].mode()[0]


In [50]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22083 entries, 0 to 22082
Data columns (total 40 columns):
 #   Column                                            Non-Null Count  Dtype  
---  ------                                            --------------  -----  
 0   Patient Age                                       22083 non-null  float64
 1   Genes in mother's side                            22083 non-null  object 
 2   Inherited from father                             22083 non-null  object 
 3   Maternal gene                                     22083 non-null  object 
 4   Paternal gene                                     22083 non-null  object 
 5   Blood cell count (mcL)                            22083 non-null  float64
 6   Mother's age                                      22083 non-null  float64
 7   Father's age                                      22083 non-null  float64
 8   Location of Institute                             22083 non-null  object 
 9   Status           

In [51]:
data['Disorder Subclass'].nunique()   

9

In [52]:
lable_y = { 'Mitochondrial genetic inheritance disorders': 0,
                  'Multifactorial genetic inheritance disorders': 1,
                  'Single-gene inheritance diseases':2
}
lable_z = {"Leber's hereditary optic neuropathy":0,
              'Cystic fibrosis':1,
              'Diabetes':2,
              'Leigh syndrome':3,
              'Cancer':4,
              'Tay-Sachs':5,
              'Hemochromatosis':6,
               'Mitochondrial myopathy':7,
                "Alzheimer's":8
 }

In [53]:
data["Genetic Disorder"] = data["Genetic Disorder"].map(lable_y)
data["Disorder Subclass"] = data["Disorder Subclass"].map(lable_z)

one hot encoding

In [54]:
m = data.columns
m

Index(['Patient Age', 'Genes in mother's side', 'Inherited from father',
       'Maternal gene', 'Paternal gene', 'Blood cell count (mcL)',
       'Mother's age', 'Father's age', 'Location of Institute', 'Status',
       'Respiratory Rate (breaths/min)', 'Heart Rate (rates/min', 'Test 1',
       'Test 2', 'Test 3', 'Test 4', 'Test 5', 'Parental consent', 'Follow-up',
       'Gender', 'Birth asphyxia',
       'Autopsy shows birth defect (if applicable)', 'Place of birth',
       'Folic acid details (peri-conceptional)',
       'H/O serious maternal illness', 'H/O radiation exposure (x-ray)',
       'H/O substance abuse', 'Assisted conception IVF/ART',
       'History of anomalies in previous pregnancies',
       'No. of previous abortion', 'Birth defects',
       'White Blood cell count (thousand per microliter)', 'Blood test result',
       'Symptom 1', 'Symptom 2', 'Symptom 3', 'Symptom 4', 'Symptom 5',
       'Genetic Disorder', 'Disorder Subclass'],
      dtype='object')

In [55]:
m = data.columns
for i in m:
    if isinstance(data[i][0],str):
        k =pd.get_dummies(data,columns=[i])
        data = pd.merge(data,k)
        data.drop(i,axis=1,inplace=True)

In [56]:
m = test.columns
for i in m:
    if isinstance(test[i][0],str):
        k =pd.get_dummies(test,columns=[i])
        test = pd.merge(test,k)
        test.drop(i,axis=1,inplace=True)

In [57]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 22083 entries, 0 to 22082
Data columns (total 96 columns):
 #   Column                                                                                                         Non-Null Count  Dtype  
---  ------                                                                                                         --------------  -----  
 0   Patient Age                                                                                                    22083 non-null  float64
 1   Blood cell count (mcL)                                                                                         22083 non-null  float64
 2   Mother's age                                                                                                   22083 non-null  float64
 3   Father's age                                                                                                   22083 non-null  float64
 4   Test 1                                        

In [ ]:
data.isnull().sum()

In [64]:
test.isnull().sum()

Patient Age                            0
Blood cell count (mcL)                 0
Mother's age                           0
Father's age                           0
Test 1                                 0
                                      ..
Blood test result_-99                  0
Blood test result_abnormal             0
Blood test result_inconclusive         0
Blood test result_normal               0
Blood test result_slightly abnormal    0
Length: 110, dtype: int64

Dividing data into three parts

In [58]:
x = data.drop(["Genetic Disorder","Disorder Subclass"],axis=1)
y = data["Genetic Disorder"]
z = data["Disorder Subclass"]

In [59]:
xtest = test

In [60]:
print(x.shape,xtest.shape)

(22083, 94) (9465, 110)


standard scaling

In [61]:
x = StandardScaler().fit_transform(x)
xtest = StandardScaler().fit_transform(xtest)

random forest

In [62]:
from sklearn.ensemble import RandomForestClassifier
rfs = RandomForestClassifier(n_estimators=500) 
rfs.fit(x, y)
rfspy = rfs.predict(xtest)  

ValueError: X has 110 features, but DecisionTreeClassifier is expecting 94 features as input.

for z parameter

In [ ]:
from sklearn.ensemble import RandomForestClassifier
lrs = RandomForestClassifier(n_estimators=500)
lrs.fit(x, z)
lrsp = lrs.predict(xtest)

In [ ]:
lable_y = { 0:'Mitochondrial genetic inheritance disorders',
                  1:'Multifactorial genetic inheritance disorders',
                 2: 'Single-gene inheritance diseases'
}
lable_z = {0:"Leber's hereditary optic neuropathy",
              1:'Cystic fibrosis',
              2:'Diabetes',
              3:'Leigh syndrome',
              4:'Cancer',
              5:'Tay-Sachs',
              6:'Hemochromatosis',
               7:'Mitochondrial myopathy',
                8:"Alzheimer's"
 }

In [ ]:
res = {
    "Patient Id":pid,
    "Genetic Disorder":rfspy,
    "Disorder Subclass":lrsp
}

In [ ]:
res = pd.DataFrame(res)

In [ ]:
res["Genetic Disorder"] = res["Genetic Disorder"].map(lable_y)
res["Disorder Subclass"] = res["Disorder Subclass"].map(lable_z)

In [ ]:
res.to_csv("submission1.csv", index=False)